# Lista 1 - Métodos Numéricos
# Professor: Cézar Santos
# Aluno: Gustavo Bulhões Carvalho da Paz Freire

## Parâmetros

In [2]:
import numpy as np
from scipy.stats import norm

In [80]:
N=9
ro=0.95
sigma=0.007
m=3

## Questão 1 - Tauchen (1986)

In [81]:
# CDF da Normal(0,sigma**2)
F = norm(loc=0, scale=sigma).cdf

# desvio padrão de y_t
std = np.sqrt(sigma**2 / (1-ro**2))

# upper bound do grid
z_N = m * std

# lower bound do grid
z_1 = - z_N

# grid
z = np.linspace(z_1, z_N, N)

# Matriz de transição
delta_z = (z_N - z_1) / (N - 1)
half_delta = 0.5 * delta_z
P = np.empty((N, N))

for i in range(N):
    P[i, 0] = F(z[0]-ro * z[i] + half_delta)
    P[i, N-1] = 1 - F(z[N-1] - ro * z[i] - half_delta)
    for j in range(1, N-1):
        x = z[j] - ro * z[i]
        P[i, j] = F(x + half_delta) - F(x - half_delta)

In [82]:
P.round(3)

array([[0.764, 0.235, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.059, 0.741, 0.2  , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.075, 0.757, 0.168, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.093, 0.767, 0.14 , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.115, 0.77 , 0.115, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.14 , 0.767, 0.093, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.168, 0.757, 0.075, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.2  , 0.741, 0.059],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.235, 0.764]])

In [83]:
z.round(4)

array([-0.0673, -0.0504, -0.0336, -0.0168,  0.    ,  0.0168,  0.0336,
        0.0504,  0.0673])

## Questão 2 - Rouwenhorst

In [84]:
# sigma_theta**2
sigma_theta2 = (sigma**2/(1 - ro**2))

# upper bound do grid
theta_N = np.sqrt((sigma_theta2)*(N-1))

# lower bound do grid
theta_1 = -theta_N

# grid
theta = np.linspace(theta_1, theta_N, N)

# p
p = (1+ro)/2

In [85]:
theta.round(4)

array([-0.0634, -0.0476, -0.0317, -0.0159,  0.    ,  0.0159,  0.0317,
        0.0476,  0.0634])

In [86]:
# P2
P2 = [[p, 1-p],[1-p, p]]
Pr = np.array(P2)
Pr

array([[0.975, 0.025],
       [0.025, 0.975]])

In [87]:
# Matriz de transição

for i in range (3,N+1):
    Pr = p*np.vstack((np.hstack((Pr,np.zeros((i-1,1)))), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.hstack((np.zeros((i-1, 1)), Pr)), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.zeros((1, i)), np.hstack((Pr, np.zeros((i-1, 1)))))) \
        + p*np.vstack((np.zeros((1, i)), np.hstack((np.zeros((i-1, 1)), Pr))))
    Pr[1:-1] /= 2.
        

In [88]:
Pr.round(3)

array([[0.817, 0.168, 0.015, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.021, 0.82 , 0.147, 0.011, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.001, 0.042, 0.823, 0.126, 0.008, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.002, 0.063, 0.825, 0.105, 0.005, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.003, 0.084, 0.825, 0.084, 0.003, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.005, 0.105, 0.825, 0.063, 0.002, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.008, 0.126, 0.823, 0.042, 0.001],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.011, 0.147, 0.82 , 0.021],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.015, 0.168, 0.817]])

## Questão 3 - Simulando para 10000 períodos e comparando

In [89]:
time = 10000

### AR(1) contínuo

In [90]:
y = np.zeros((1,time))
cont = np.transpose(y)

In [91]:
# epslon (normal padrão)
e = np.random.normal(loc=0, scale=1, size=time)
cont[0] = 0

In [92]:
# simulando
for t in range(1,time):
    cont[t] = ro*cont[t-1] + sigma*e[t]

In [93]:
# criando variável tempo
tempo = np.linspace(1,time,time, dtype=np.int64)

### AR(1) discretizado - Tauchen

In [94]:
tauch = np.ones((time,1), dtype = np.int64)

In [95]:
# simulando
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= P[tauch[t-1]].cumsum())
    g = np.transpose(x)
    tauch[t] = g[0]

In [96]:
# recuperando os valores no grid
tauchen = z[tauch]

### AR(1) discretizado - Rouwenhorst

In [97]:
rouwen = np.ones((time,1), dtype = np.int64)

In [98]:
# simulando
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= Pr[rouwen[t-1]].cumsum())
    g = np.transpose(x)
    rouwen[t] = g[0]

In [99]:
# recuperando os valores no grid
rouwenhorst = theta[rouwen]

### Comparativo

In [101]:
import plotly.plotly as py
import plotly.graph_objs as go

In [102]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)
trace2 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen vs rouwenhorst')

Plotando os 3 processos juntos, vemos que em geral as simulações dos métodos discretizados estão razoavelmente próximas do AR(1) contínuo. Vamos comparar agora um processo discretizado de cada vez. 

In [103]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen')

In [104]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs rouwenhorst')

Nota-se que, para os valores dados dos parâmetros, o método de Tauchen parece aproximar melhor o processo contínuo do que o de Rouwenhorst. Para vermos em mais detalhes, vamos analisar especificamente para as 1000 primeiras simulações.

In [105]:
# apenas 1000 primeiros períodos
tempo2 = np.linspace(1, 1000, 1000, dtype = np.int64)

In [106]:
# gerando vetores para primeiros 1000 períodos
cont1000 = cont[0:1000]
tauchen1000 = tauchen[0:1000]
rouwenhorst1000 = rouwenhorst[0:1000]

In [107]:
trace0 = go.Scatter(
    x = tempo2,
    y = cont1000,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo2,
    y = tauchen1000,
    mode = 'lines',
    name = 'Tauchen'
)
trace2 = go.Scatter(
    x = tempo2,
    y = rouwenhorst1000,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    title='Comparativo das 1000 primeiras simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen vs rouwenhorst 1000')

No entanto, nas 1000 primeiras simulações o método de Rouwenhorst parece ter performado melhor que o de Tauchen.

## Questão 4 - Estimando AR(1) com base nos dados simulados

In [31]:
import scipy as sp

### Tauchen

In [109]:
# ajustando dimensões
x = np.transpose(tauchen[0:-1])
xp = x.ravel()
w = np.transpose(tauchen[1:])
wp = w.ravel()

In [110]:
# estimando rô
h = sp.polyfit(xp,wp,1)
ro_hat_tauchen = h[0]
ro_hat_tauchen

0.9515206315244659

In [111]:
# estimando sigma
wp_hat = sp.polyval(h,xp)
variance_t = sp.var(wp - wp_hat)
sigma_hat_tauchen = sp.sqrt(variance_t)
sigma_hat_tauchen

0.008015229719480228

### Rouwenhorst

In [112]:
# ajustando dimensões
a = np.transpose(rouwenhorst[0:-1])
ap = a.ravel()
b = np.transpose(rouwenhorst[1:])
bp = b.ravel()

In [113]:
# estimando rô
g = sp.polyfit(ap,bp,1)
ro_hat_rouwenhorst = g[0]
ro_hat_rouwenhorst

0.949216730373561

In [114]:
# estimando sigma
bp_hat = sp.polyval(g,ap)
variance_r = sp.var(bp - bp_hat)
sigma_hat_rouwenhorst = sp.sqrt(variance_r)
sigma_hat_rouwenhorst

0.006907035193957205

Estimando processos AR(1) com base nos dados simulados, notamos que ambos estão próximos do processo gerador de dados real, com os coeficientes estimados com os dados simulados do método de Rouwenhorst chegando mais perto dos parâmetros verdadeiros. 

### Questão 5 - Refazendo questões acima para rô = 0.99

In [115]:
ro=0.99

#### Questão 1

In [116]:
# CDF da Normal(0,sigma**2)
F = norm(loc=0, scale=sigma).cdf

# desvio padrão de y_t
std = np.sqrt(sigma**2 / (1-ro**2))

# upper bound do grid
z_N = m * std

# lower bound do grid
z_1 = - z_N

# grid
z = np.linspace(z_1, z_N, N)

# Matriz de transição
delta_z = (z_N - z_1) / (N - 1)
half_delta = 0.5 * delta_z
P = np.empty((N, N))

for i in range(N):
    P[i, 0] = F(z[0]-ro * z[i] + half_delta)
    P[i, N-1] = 1 - F(z[N-1] - ro * z[i] - half_delta)
    for j in range(1, N-1):
        x = z[j] - ro * z[i]
        P[i, j] = F(x + half_delta) - F(x - half_delta)

In [117]:
P.round(3)

array([[0.993, 0.007, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.002, 0.991, 0.006, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.003, 0.992, 0.005, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.003, 0.992, 0.005, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.004, 0.992, 0.004, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.005, 0.992, 0.003, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.005, 0.992, 0.003, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.006, 0.991, 0.002],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.007, 0.993]])

In [118]:
z.round(4)

array([-0.1489, -0.1116, -0.0744, -0.0372,  0.    ,  0.0372,  0.0744,
        0.1116,  0.1489])

#### Questão 2

In [119]:
# sigma_theta**2
sigma_theta2 = (sigma**2/(1 - ro**2))

# upper bound do grid
theta_N = np.sqrt((sigma_theta2)*(N-1))

# lower bound do grid
theta_1 = -theta_N

# grid
theta = np.linspace(theta_1, theta_N, N)

# p
p = (1+ro)/2

In [120]:
theta.round(4)

array([-0.1404, -0.1053, -0.0702, -0.0351,  0.    ,  0.0351,  0.0702,
        0.1053,  0.1404])

In [121]:
# P2
P2 = [[p, 1-p],[1-p, p]]
Pr = np.array(P2)
Pr

array([[0.995, 0.005],
       [0.005, 0.995]])

In [122]:
# Matriz de transição

for i in range (3,N+1):
    Pr = p*np.vstack((np.hstack((Pr,np.zeros((i-1,1)))), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.hstack((np.zeros((i-1, 1)), Pr)), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.zeros((1, i)), np.hstack((Pr, np.zeros((i-1, 1)))))) \
        + p*np.vstack((np.zeros((1, i)), np.hstack((np.zeros((i-1, 1)), Pr))))
    Pr[1:-1] /= 2.
        

In [123]:
Pr.round(3)

array([[0.961, 0.039, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.005, 0.961, 0.034, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.01 , 0.961, 0.029, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.014, 0.961, 0.024, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.019, 0.961, 0.019, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.024, 0.961, 0.014, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.029, 0.961, 0.01 , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.034, 0.961, 0.005],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.039, 0.961]])

#### Questão 3

In [124]:
# simulando - contínuo
for t in range(1,time):
    cont[t] = ro*cont[t-1] + sigma*e[t]

In [125]:
# simulando - tauchen
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= P[tauch[t-1]].cumsum())
    g = np.transpose(x)
    tauch[t] = g[0]

In [126]:
# recuperando os valores no grid
tauchen = z[tauch]

In [127]:
# simulando - rouwenhorst
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= Pr[rouwen[t-1]].cumsum())
    g = np.transpose(x)
    rouwen[t] = g[0]

In [128]:
# recuperando os valores no grid
rouwenhorst = theta[rouwen]

In [129]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)
trace2 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen vs rouwenhorst')

In [130]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen')

In [131]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs rouwenhorst')

Podemos observar que com rô=0.99 a performance dos métodos piorou consideravelmente. Porém, como esperado pelo valor de rô ser próximo de 1, o método de Rouwenhorst aproxima melhor o processo contínuo do que o de Tauchen. 

#### Questão 4

In [132]:
# ajustando dimensões
x = np.transpose(tauchen[0:-1])
xp = x.ravel()
w = np.transpose(tauchen[1:])
wp = w.ravel()

# estimando rô
h = sp.polyfit(xp,wp,1)
ro_hat_tauchen = h[0]
ro_hat_tauchen

0.9980719725511991

In [133]:
# estimando sigma
wp_hat = sp.polyval(h,xp)
variance_t = sp.var(wp - wp_hat)
sigma_hat_tauchen = sp.sqrt(variance_t)
sigma_hat_tauchen

0.0033685391580913953

In [134]:
# ajustando dimensões
a = np.transpose(rouwenhorst[0:-1])
ap = a.ravel()
b = np.transpose(rouwenhorst[1:])
bp = b.ravel()

# estimando rô
g = sp.polyfit(ap,bp,1)
ro_hat_rouwenhorst = g[0]
ro_hat_rouwenhorst

0.9897898464304434

In [135]:
# estimando sigma
bp_hat = sp.polyval(g,ap)
variance_r = sp.var(bp - bp_hat)
sigma_hat_rouwenhorst = sp.sqrt(variance_r)
sigma_hat_rouwenhorst

0.0071980372761308985

A superioridade do método de Rouwenhorst para rô=0.99 é confirmada pelas estimações baseadas nos dados simulados. Enquanto as estimações baseadas nos dados do método de Rouwenhorst chegam bem próximas aos valores dos parâmetros, o sigma estimado com dados do método de Tauchen está distante do verdadeiro.

### Questão 5 - Grid com 20 pontos

Como as simulações não ficaram próximas do processo contínuo e os resultados estimados de Tauchen foram ruins, vamos refazer as questões para um grid com 20 pontos e comparar os resultados.

In [136]:
N=20

#### Questão 1

In [137]:
# CDF da Normal(0,sigma**2)
F = norm(loc=0, scale=sigma).cdf

# desvio padrão de y_t
std = np.sqrt(sigma**2 / (1-ro**2))

# upper bound do grid
z_N = m * std

# lower bound do grid
z_1 = - z_N

# grid
z = np.linspace(z_1, z_N, N)

# Matriz de transição
delta_z = (z_N - z_1) / (N - 1)
half_delta = 0.5 * delta_z
P = np.empty((N, N))

for i in range(N):
    P[i, 0] = F(z[0]-ro * z[i] + half_delta)
    P[i, N-1] = 1 - F(z[N-1] - ro * z[i] - half_delta)
    for j in range(1, N-1):
        x = z[j] - ro * z[i]
        P[i, j] = F(x + half_delta) - F(x - half_delta)

In [138]:
P.round(3)

array([[0.818, 0.181, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.095, 0.728, 0.176, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.099, 0.73 , 0.17 , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.103, 0.732, 0.164, 0.001, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.107, 0.733, 0.159, 0.001, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.111, 0.735, 0.154, 0.001, 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.

In [139]:
z.round(4)

array([-0.1489, -0.1332, -0.1175, -0.1019, -0.0862, -0.0705, -0.0548,
       -0.0392, -0.0235, -0.0078,  0.0078,  0.0235,  0.0392,  0.0548,
        0.0705,  0.0862,  0.1019,  0.1175,  0.1332,  0.1489])

#### Questão 2

In [140]:
# sigma_theta**2
sigma_theta2 = (sigma**2/(1 - ro**2))

# upper bound do grid
theta_N = np.sqrt((sigma_theta2)*(N-1))

# lower bound do grid
theta_1 = -theta_N

# grid
theta = np.linspace(theta_1, theta_N, N)

# p
p = (1+ro)/2

In [141]:
theta.round(4)

array([-0.2163, -0.1935, -0.1708, -0.148 , -0.1252, -0.1025, -0.0797,
       -0.0569, -0.0342, -0.0114,  0.0114,  0.0342,  0.0569,  0.0797,
        0.1025,  0.1252,  0.148 ,  0.1708,  0.1935,  0.2163])

In [142]:
# P2
P2 = [[p, 1-p],[1-p, p]]
Pr = np.array(P2)
Pr

array([[0.995, 0.005],
       [0.005, 0.995]])

In [143]:
# Matriz de transição

for i in range (3,N+1):
    Pr = p*np.vstack((np.hstack((Pr,np.zeros((i-1,1)))), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.hstack((np.zeros((i-1, 1)), Pr)), np.zeros((1, i)))) \
        + (1-p)*np.vstack((np.zeros((1, i)), np.hstack((Pr, np.zeros((i-1, 1)))))) \
        + p*np.vstack((np.zeros((1, i)), np.hstack((np.zeros((i-1, 1)), Pr))))
    Pr[1:-1] /= 2.
        

In [144]:
Pr.round(3)

array([[0.909, 0.087, 0.004, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.005, 0.91 , 0.082, 0.004, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.009, 0.91 , 0.078, 0.003, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.014, 0.91 , 0.073, 0.003, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.018, 0.911, 0.069, 0.002, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.023, 0.911, 0.064, 0.002, 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.

#### Questão 3

In [145]:
# simulando - contínuo
for t in range(1,time):
    cont[t] = ro*cont[t-1] + sigma*e[t]

In [146]:
# simulando - tauchen
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= P[tauch[t-1]].cumsum())
    g = np.transpose(x)
    tauch[t] = g[0]

In [147]:
# recuperando os valores no grid
tauchen = z[tauch]

In [148]:
# simulando - rouwenhorst
for t in range(2,time):
    x = np.where(norm(loc=0, scale=1).cdf(e[t]) <= Pr[rouwen[t-1]].cumsum())
    g = np.transpose(x)
    rouwen[t] = g[0]

In [149]:
# recuperando os valores no grid
rouwenhorst = theta[rouwen]

In [150]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)
trace2 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen vs rouwenhorst')

In [151]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = tauchen,
    mode = 'lines',
    name = 'Tauchen'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs tauchen')

In [152]:
trace0 = go.Scatter(
    x = tempo,
    y = cont,
    mode = 'lines',
    name = 'Contínuo'
)
trace1 = go.Scatter(
    x = tempo,
    y = rouwenhorst,
    mode = 'lines',
    name = 'Rouwenhorst'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Comparativo das simulações do AR(1) contínuo e discretizado',
    xaxis=dict(
        title='Tempo'
        )
    ,
    yaxis=dict(
        title='Processo'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cont vs rouwenhorst')

Com 20 pontos no grid, o método de Tauchen melhorou consideravelmente e parece aproximar melhor o processo contínuo do que o de Rouwenhorst.

#### Questão 4

In [153]:
# ajustando dimensões
x = np.transpose(tauchen[0:-1])
xp = x.ravel()
w = np.transpose(tauchen[1:])
wp = w.ravel()

# estimando rô
h = sp.polyfit(xp,wp,1)
ro_hat_tauchen = h[0]
ro_hat_tauchen

0.9887724574028535

In [154]:
# estimando sigma
wp_hat = sp.polyval(h,xp)
variance_t = sp.var(wp - wp_hat)
sigma_hat_tauchen = sp.sqrt(variance_t)
sigma_hat_tauchen

0.007953787122432178

In [155]:
# ajustando dimensões
a = np.transpose(rouwenhorst[0:-1])
ap = a.ravel()
b = np.transpose(rouwenhorst[1:])
bp = b.ravel()

# estimando rô
g = sp.polyfit(ap,bp,1)
ro_hat_rouwenhorst = g[0]
ro_hat_rouwenhorst

0.9908169803761103

In [156]:
# estimando sigma
bp_hat = sp.polyval(g,ap)
variance_r = sp.var(bp - bp_hat)
sigma_hat_rouwenhorst = sp.sqrt(variance_r)
sigma_hat_rouwenhorst

0.007051988228267814

Com 20 pontos no grid, as estimações baseadas nos dados simulados pelo método de Tauchen voltaram a ficar próximas, mas Rouwenhorst continua performando melhor nesse quesito, estando mais próximo dos valores reais dos parâmetros. 